<a href="https://colab.research.google.com/github/jny0812/trading_project/blob/main/%08trading_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1) 모듈 import

In [ ]:
!pip3 install -U finance-datareader
!pip3 install pykrx

In [ ]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from pykrx import stock
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

## 크롤링

In [ ]:
stockList = fdr.StockListing('KRX')

def crawling(page, kospi100):
    res = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(page))
    soup = BeautifulSoup(res.content,'html.parser')
    
    #section = soup.select('tbody')
    #items= section.find_all('tr', onmouseover="mouseOver(this)")
    
    items = soup.select('tbody > tr > td:nth-of-type(2) > a')
    
    for item in range(len(items)):
        name=items[item].contents[0]
        if (stockList[stockList['Name']==name]['ListingDate']<'2018-1-1').tolist():
            code=items[item].attrs['href'][-6:]
            kospi100[code]=name
            
            if len(kospi100)==100:
                break
                
    return kospi100

def Kospi_100(): #kospi 상위 100개 출력 함수
    
    page = 1
    kospi100=dict()
    
    while(len(kospi100)!=100):  
        kospi100=crawling(page,kospi100)
        page=page+1
        
    return kospi100


In [ ]:
kospi100 = Kospi_100()

kospi100

In [ ]:
kospi_df = pd.DataFrame()

kospi_df['Code'] = kospi100.keys()
kospi_df['Name'] =kospi100.values()

kospi_df

### 3) Trading 

- 특정 종목의 정보를 가져오는 함수 trading() 생성

In [ ]:

def trading(code,start_date,end_date):
    df = fdr.DataReader(code,start_date, end_date)

    return df

In [ ]:

#1) before corona

for i in range(len(kospi_df)):
    code = kospi_df['Code'][i]
    df_kospi = trading(code,'20180101','20191231')
    
df_kospi

###   * Strategy 
-  주가가 20일 이평선을 넘어가면 매수
- 주가 하락하는 시점에 바로 매도 (100주) 

- initial = 1억

In [ ]:
#삼성

se_df = trading('005930','20180101','20191231')

#### 데이터 전처리

- 1. 결측치 제거
- 2. 회귀분석
- 3. 상관분석..?

In [ ]:

se_df = se_df.dropna()

###  전략: 주가가 20일 이평선을 넘어가면 매수 & 주가 하락하는 시점에 바로 매도 (100주) & initial = 1억


In [ ]:
se_df.head()

In [ ]:
# plot 하기 위해

# 데이터의 type을 int로 바꿔줌
se_df[['Open','High','Low','Volume']] = se_df[['Open','High','Low','Volume']].astype(int) 


- 이상치 확인

In [ ]:
se_df['Close'].plot()

In [ ]:
plt.figure(figsize=(12, 3))
plt.scatter(se_df.index, se_df['Open'],color='green',alpha=0.8,linewidth=1,label='Open')
plt.legend()
plt.show()

In [ ]:
se_2018 = trading('005930','20180401','20180531')
se_2018 = se_2018.dropna()
se_2018['MA_20'] = se_2018['Close'].rolling(20).mean()
se_2018

In [ ]:
outlier = se_df[se_df.index == '2018-04-30'].index
se_df = se_df.drop(outlier,axis=0)

In [ ]:
outlier = se_df[se_df.index == '2018-05-02'].index
se_df = se_df.drop(outlier,axis=0)

In [ ]:
outlier = se_df[se_df.index == '2018-05-03'].index
se_df = se_df.drop(outlier,axis=0)

In [ ]:
plt.figure(figsize=(12, 3))
plt.scatter(se_df.index, se_df['Open'],color='green',alpha=0.8,linewidth=1,label='Open')
plt.legend()
plt.show()

### => '2018-04-31' ~ '2018-05-03'  데이터에 문제가 있으므로 제거

In [ ]:
type(se_df.index)


- **buy_signal이 0에서 1로 넘어가는 지점에서 매수**
- **sell_signal이 -1.0 이면 매도**

In [ ]:
#se.resample('M').sum()

se_df['MA_20'] = se_df['Close'].rolling(20).mean()

In [ ]:
#이동평균값 plot 그리기
ma_plot = se_df[['MA_20','Close']].plot(figsize=(10,6))

ma_plot.plot()

ma_plot.legend(loc='best') #라벨 위치 설정

ma_plot.grid() #격자

In [ ]:
se_df['buy_signal'] = np.where(se_df['MA_20'] > se_df['Close'], 1.0, 0.0)
se_df['buy_signal'] = se_df['buy_signal'].diff()

In [ ]:
se_df['sell_signal'] = 0.0

In [ ]:
for i in range(1,len(se_df)):
    if se_df['Close'][i-1] > se_df['Close'][i]:
        se_df['sell_signal'][i] = -1.0
    else:
        se_df['sell_signal'][i] = 0.0

In [ ]:
fig = plt.figure(figsize=(30,10))
ax = fig.add_subplot(111)
index = se_df.index.astype('str') # 캔들스틱 x축이 str로 들어감

# 이동평균선 그리기
ax.plot(index, se_df['MA_20'], label='MA_20', linewidth=0.7)

# X축 티커 숫자 20개로 제한
ax.xaxis.set_major_locator(ticker.MaxNLocator(20))

# 그래프 title과 축 이름 지정
ax.set_title('samsung', fontsize=22)
ax.set_xlabel('Date')

# 캔들차트 그리기
candlestick2_ohlc(ax, se_df['Open'], se_df['High'], 
                  se_df['Low'], se_df['Close'],
                  width=0.5, colorup='r', colordown='b')
ax.legend()
plt.grid()
plt.show()